<a href="https://colab.research.google.com/github/blinders79/product-search/blob/main/CustomerSentiSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade openai httpx
!pip install --upgrade openai
!pip install httpx==0.23.0 # Downgrade httpx to a compatible version
!pip install python-dotenv

import pandas as pd
import numpy as np
import os
from openai import OpenAI
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity



# Function to generate customer descriptions using OpenAI
def generate_customer_description(row):

      # Loading API key and organization ID from a dotenv file
    load_dotenv(dotenv_path='apikey.env.txt')
    # Retrieving API key and organization ID from environment variables
    APIKEY = os.getenv("APIKEY")
    # Set your API key
    client = OpenAI(api_key=APIKEY)
    prompt = f"""
    Generate a concise description for a customer based on the following information:
    Name: {row['Name']}
    Category: {row['Category']}
    Age: {row['Age']}
    Gender: {row['Gender']}
    Income: {row['Income Level']}
    Income: {row['Education']}
    Income: {row['Occupation']}
    Income: {row['Dependents']}
    """

    completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content


# Function to perform sentiment analysis using OpenAI
def analyze_sentiment(review):
   # Loading API key and organization ID from a dotenv file
    load_dotenv(dotenv_path='apikey.env.txt')
    # Retrieving API key and organization ID from environment variables
    APIKEY = os.getenv("APIKEY")
    # Set your API key
    client = OpenAI(api_key=APIKEY)
    completion = client.chat.completions.create(  # Updated to use openai.ChatCompletion.create
        model="gpt-4", # Use a newer model like gpt-3.5-turbo
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"What is the sentiment of the following review: {review}"}
        ] # Adjust as needed
    )
    return completion.choices[0].message.content


def filter_customer_data(name_or_partial_name):
  df_customer = pd.read_csv('customer_info_final.csv')
  display(df_customer.head())

  duplicate_reviews = df_customer[df_customer['Review'].duplicated(keep=False)]
  print("\nDuplicate Reviews:")
  display(duplicate_reviews[['Name', 'Review']])

  df_customer_no_duplicates = df_customer.drop_duplicates(subset=['Review'], keep='first')
  print(f"Shape of the DataFrame after removing duplicate reviews: {df_customer_no_duplicates.shape}")

  # Load the customer data
  df_customer = pd.read_csv('customer_info_final.csv')
  df_customer.drop_duplicates(subset=['Review'], keep='first', inplace=True)

  # Apply the function to create customer descriptions
  filtered_df = df_customer[df_customer['Name'].str.contains(name_or_partial_name, case=False)]
  filtered_df['customer_description'] = filtered_df.apply(generate_customer_description, axis=1)



  # Apply the function to analyze sentiment
  filtered_df['sentiment'] = filtered_df['Review'].apply(analyze_sentiment)

  #filtered_df = df_customer[df_customer['Name'].str.contains(name_or_partial_name, case=False)]
  # Save the updated data to a JSON file
  #customer_data_json = df_customer[['Name', 'sentiment', 'customer_description']].to_json(orient='records')
  #with open('customer_data.json', 'w') as f:
    #f.write(customer_data_json)
  return filtered_df

def generate_chatgpt_response(user_input):
  # Loading API key and organization ID from a dotenv file
  load_dotenv(dotenv_path='apikey.env.txt')
  # Retrieving API key and organization ID from environment variables
  APIKEY = os.getenv("APIKEY")
  # Set your API key
  client = OpenAI(api_key=APIKEY)
  #print(top_responses)

  prompt = f"User: {user_input}\n"
  prompt += "Generate a response using **only** the information provided above. Do not include any external knowledge or assumptions."  # Emphasize restriction

  completion = client.chat.completions.create(
      model="gpt-4",
      messages=[
          {"role": "system", "content": "You are a helpful AI assistant that provides responses based solely on provided information."},  # Set system role
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": f"summarize the content in third person tone based on :\n1. {user_input}"}
      ]
    )

  #print(completion.choices[0])

  # Try this first
  # return completion.choices[0].message.content

  # If above line fails, try this
  return completion.choices[0].message.content

def get_sentiment(name_or_partial_name):

  filtered_df = filter_customer_data(name_or_partial_name)
  return generate_chatgpt_response(filtered_df[['Name', 'sentiment', 'Review',"Category"]].to_dict(orient='records'))

def get_customer_info(name_or_partial_name):
  filtered_df = filter_customer_data(name_or_partial_name)
  return generate_chatgpt_response(filtered_df[['Name', 'sentiment', 'Review',"Category",'customer_description']].to_dict(orient='records'))



  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.12.0
    Uninstalling h11-0.12.0:
      Successfully uninstalled h11-0.12.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.15.0
    Uninstalling httpcore-0.15.0:
      Successfully uninstalled httpcore-0.15.0
  Attempting uninstall: httpx
    Found existing installation: httpx 0.23.0
    Uninstalling httpx-0.23.0:
      Successfully uninstalled httpx-0.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.7.0 requires

In [ ]:
!pip install flask flask-ngrok
!pip install fastapi uvicorn nest-asyncio
!pip install flask pyngrok
!ngrok authtoken "2uoW1dUut1ruqCm63wSUXxukjIP_3676oEoZefzqpEgqTW5AL"

from flask import Flask, request, jsonify
from pyngrok import ngrok
import pandas as pd

# Create Flask app
app = Flask(__name__)

customer_names = pd.read_csv('customer_info_final.csv')['Name'].tolist()

@app.route("/")
def home():
    return "API is running!"

@app.route("/sentiment", methods=["POST"])
def get_no_senti():
    data = request.get_json()
    input_text = data.get("prompt")

    # Find the name from the customer list in the prompt
    name = next((name for name in customer_names if name in input_text), None)

    if name:
        responseData = get_sentiment(name)
        return responseData
    else:
        return jsonify({"error": "Name not found in customer list"}), 400

@app.route("/customer_info", methods=["POST"])
def get_cust_info():
    data = request.get_json()
    input_text = data.get("prompt")

    # Find the name from the customer list in the prompt
    name = next((name for name in customer_names if name in input_text), None)

    if name:
        responseData = get_customer_info(name)
        return responseData
    else:
        return jsonify({"error": "Name not found in customer list"}), 400



# Start the Flask app
public_url = ngrok.connect(5000).public_url
print(f"Public URL: {public_url}")

app.run(port=5000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: https://6c45-34-86-205-239.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


,Unnamed: 0,Customer ID,Age,Gender,Marital Status,Income Level,Education,Occupation,Residential Status,Dependents,...,Credit_Bureau,last_3months_personal_loan_inq,last_3months_credit_card_inq,last_3months_mortgage_inq,last_6months_personal_loan_inq,last_6months_credit_card_inq,last_6months_mortgage_inq,Name,Review,Category
0,0,3d06c8d0-ebc7-45fd-bfaf-e10fcfed1c7c,56,Male,Divorced,High,University,"Psychotherapist, child",Owns house,"(0,)",...,801,False,True,True,False,True,False,Quinn Lee,I love their credit card rewards program. Cash...,Luxury
1,1,29ff97c8-7c46-4da7-9b5b-3aa8dde8da5b,56,Male,Single,Medium,University,Diplomatic Services operational officer,Living with parents,"(3,)",...,835,False,True,True,False,True,False,Paul Taylor,"Investment options are diverse, but I'd apprec...",Luxury
2,2,bb43e090-ba19-4d5c-91d7-e500612d92b9,51,Female,Widowed,Low,College,Hydrographic surveyor,Rents,"(3,)",...,796,True,False,True,False,False,True,Noah Martin,I appreciate the bank's strong fraud protectio...,Mid Income
3,3,7d431b5f-5c67-4073-af3f-4eb2b574852f,29,Male,Single,Low,College,Housing manager/officer,Rents,"(4,)",...,810,False,True,True,True,False,False,Emma Johnson,More competitive foreign exchange rates would ...,Mid Income
4,4,9dbac557-be11-4a84-934f-9d5a531f8d34,44,Male,Divorced,High,University,"Teacher, music",Living with parents,"(2,)",...,790,True,False,True,True,True,False,Noah Rodriguez,"Their mortgage plans are transparent, and the ...",Mid Income



Duplicate Reviews:


,Name,Review
0,Quinn Lee,I love their credit card rewards program. Cash...
1,Paul Taylor,"Investment options are diverse, but I'd apprec..."
2,Noah Martin,I appreciate the bank's strong fraud protectio...
3,Emma Johnson,More competitive foreign exchange rates would ...
4,Noah Rodriguez,"Their mortgage plans are transparent, and the ..."
5,Tina Taylor,They offer a great range of financial products...
6,Grace Scott,Competitive interest rates and excellent custo...
7,Alice White,"Investment options are diverse, but I'd apprec..."
8,Isla Parker,Competitive interest rates and excellent custo...
9,Rachel Martin,I love their credit card rewards program. Cash...


...

,Unnamed: 0,Customer ID,Age,Gender,Marital Status,Income Level,Education,Occupation,Residential Status,Dependents,...,Credit_Bureau,last_3months_personal_loan_inq,last_3months_credit_card_inq,last_3months_mortgage_inq,last_6months_personal_loan_inq,last_6months_credit_card_inq,last_6months_mortgage_inq,Name,Review,Category
0,0,3d06c8d0-ebc7-45fd-bfaf-e10fcfed1c7c,56,Male,Divorced,High,University,"Psychotherapist, child",Owns house,"(0,)",...,801,False,True,True,False,True,False,Quinn Lee,I love their credit card rewards program. Cash...,Luxury
1,1,29ff97c8-7c46-4da7-9b5b-3aa8dde8da5b,56,Male,Single,Medium,University,Diplomatic Services operational officer,Living with parents,"(3,)",...,835,False,True,True,False,True,False,Paul Taylor,"Investment options are diverse, but I'd apprec...",Luxury
2,2,bb43e090-ba19-4d5c-91d7-e500612d92b9,51,Female,Widowed,Low,College,Hydrographic surveyor,Rents,"(3,)",...,796,True,False,True,False,False,True,Noah Martin,I appreciate the bank's strong fraud protectio...,Mid Income
3,3,7d431b5f-5c67-4073-af3f-4eb2b574852f,29,Male,Single,Low,College,Housing manager/officer,Rents,"(4,)",...,810,False,True,True,True,False,False,Emma Johnson,More competitive foreign exchange rates would ...,Mid Income
4,4,9dbac557-be11-4a84-934f-9d5a531f8d34,44,Male,Divorced,High,University,"Teacher, music",Living with parents,"(2,)",...,790,True,False,True,True,True,False,Noah Rodriguez,"Their mortgage plans are transparent, and the ...",Mid Income



Duplicate Reviews:


,Name,Review
0,Quinn Lee,I love their credit card rewards program. Cash...
1,Paul Taylor,"Investment options are diverse, but I'd apprec..."
2,Noah Martin,I appreciate the bank's strong fraud protectio...
3,Emma Johnson,More competitive foreign exchange rates would ...
4,Noah Rodriguez,"Their mortgage plans are transparent, and the ..."
5,Tina Taylor,They offer a great range of financial products...
6,Grace Scott,Competitive interest rates and excellent custo...
7,Alice White,"Investment options are diverse, but I'd apprec..."
8,Isla Parker,Competitive interest rates and excellent custo...
9,Rachel Martin,I love their credit card rewards program. Cash...


Shape of the DataFrame after removing duplicate reviews: (17, 21)


<ipython-input-1-b9c0c6ad07cb>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['customer_description'] = filtered_df.apply(generate_customer_description, axis=1)
<ipython-input-1-b9c0c6ad07cb>:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sentiment'] = filtered_df['Review'].apply(analyze_sentiment)
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 14:37:11] "POST /customer_info HTTP/1.1" 200 -
